In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

### Create Vine Dataframe for analysis

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [5]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter df where total_votes count is equal to or greater than 20
total_votes_gt_20_df = vine_df.filter(col("total_votes")>= 20)
total_votes_gt_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [6]:
# Dataframe containing helpful votes more than 50%
helpful_votes_gt_50pc = total_votes_gt_20_df.filter(col("helpful_votes")/col("total_votes")>= 0.5)
helpful_votes_gt_50pc.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [11]:
# Filter Dataframe where vine = Y
paid_reviews_df = helpful_votes_gt_50pc.filter(col("vine")== 'Y')
paid_reviews_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [12]:
# Filter Dataframe where vine = N
unpaid_reviews_df = helpful_votes_gt_50pc.filter(col("vine")== 'N')
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [28]:
# PAID REVIEWS - Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
total_paid_reviews = paid_reviews_df.count()
print("Total number of paid reviews = %f" %total_paid_reviews)

total_paid_5_star_reviews = paid_reviews_df.filter(col("star_rating") == 5).count()
print("Total number of paid 5 star reviews = %f" %total_paid_5_star_reviews)

paid_5_star_review_percentage = total_paid_5_star_reviews/total_paid_reviews
print("Percentage of paid 5 star reviews = %f" %paid_5_star_review_percentage)

Total number of paid reviews = 60.000000
Total number of paid 5 star reviews = 34.000000
Percentage of paid 5 star reviews = 0.566667


In [29]:
# UNPAID REVIEWS - Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
total_unpaid_reviews = unpaid_reviews_df.count()
print("Total number of unpaid reviews = %f" %total_unpaid_reviews)

total_unpaid_5_star_reviews = unpaid_reviews_df.filter(col("star_rating") == 5).count()
print("Total number of unpaid 5 star reviews = %f" %total_unpaid_5_star_reviews)

unpaid_5_star_review_percentage = total_unpaid_5_star_reviews/total_unpaid_reviews
print("Percentage of unpaid 5 star reviews = %f" %unpaid_5_star_review_percentage)

Total number of unpaid reviews = 14477.000000
Total number of unpaid 5 star reviews = 8212.000000
Percentage of unpaid 5 star reviews = 0.567245


In [30]:
# 4-star paid reviews
total_paid_4_star_reviews = paid_reviews_df.filter(col("star_rating") == 4).count()
print("Total number of paid 4 star reviews = %f" %total_paid_4_star_reviews)

paid_4_star_review_percentage = total_paid_4_star_reviews/total_paid_reviews
print("Percentage of paid 4 star reviews = %f" %paid_4_star_review_percentage)

Total number of paid 4 star reviews = 16.000000
Percentage of paid 4 star reviews = 0.266667


In [32]:
# 4-star unpaid reviews
total_unpaid_4_star_reviews = unpaid_reviews_df.filter(col("star_rating") == 4).count()
print("Total number of unpaid 4 star reviews = %f" %total_unpaid_4_star_reviews)

unpaid_4_star_review_percentage = total_unpaid_4_star_reviews/total_unpaid_reviews
print("Percentage of unpaid 4 star reviews = %f" %unpaid_4_star_review_percentage)

Total number of unpaid 4 star reviews = 2687.000000
Percentage of unpaid 4 star reviews = 0.185605


In [33]:
# 3-star paid reviews
total_paid_3_star_reviews = paid_reviews_df.filter(col("star_rating") == 3).count()
print("Total number of paid 3 star reviews = %f" %total_paid_3_star_reviews)

paid_3_star_review_percentage = total_paid_3_star_reviews/total_paid_reviews
print("Percentage of paid 3 star reviews = %f" %paid_3_star_review_percentage)

Total number of paid 3 star reviews = 9.000000
Percentage of paid 3 star reviews = 0.150000


In [34]:
# 3-star unpaid reviews
total_unpaid_3_star_reviews = unpaid_reviews_df.filter(col("star_rating") == 3).count()
print("Total number of unpaid 3 star reviews = %f" %total_unpaid_3_star_reviews)

unpaid_3_star_review_percentage = total_unpaid_3_star_reviews/total_unpaid_reviews
print("Percentage of unpaid 3 star reviews = %f" %unpaid_3_star_review_percentage)

Total number of unpaid 3 star reviews = 1292.000000
Percentage of unpaid 3 star reviews = 0.089245


In [35]:
# 2-star paid reviews
total_paid_2_star_reviews = paid_reviews_df.filter(col("star_rating") == 2).count()
print("Total number of paid 2 star reviews = %f" %total_paid_2_star_reviews)

paid_2_star_review_percentage = total_paid_2_star_reviews/total_paid_reviews
print("Percentage of paid 2 star reviews = %f" %paid_2_star_review_percentage)

Total number of paid 2 star reviews = 1.000000
Percentage of paid 2 star reviews = 0.016667


In [36]:
# 2-star unpaid reviews
total_unpaid_2_star_reviews = unpaid_reviews_df.filter(col("star_rating") == 2).count()
print("Total number of unpaid 2 star reviews = %f" %total_unpaid_2_star_reviews)

unpaid_2_star_review_percentage = total_unpaid_2_star_reviews/total_unpaid_reviews
print("Percentage of unpaid 2 star reviews = %f" %unpaid_2_star_review_percentage)

Total number of unpaid 2 star reviews = 754.000000
Percentage of unpaid 2 star reviews = 0.052083


In [37]:
# 1-star paid reviews
total_paid_1_star_reviews = paid_reviews_df.filter(col("star_rating") == 1).count()
print("Total number of paid 1 star reviews = %f" %total_paid_1_star_reviews)

paid_1_star_review_percentage = total_paid_1_star_reviews/total_paid_reviews
print("Percentage of paid 1 star reviews = %f" %paid_1_star_review_percentage)

Total number of paid 1 star reviews = 0.000000
Percentage of paid 1 star reviews = 0.000000


In [38]:
# 1-star unpaid reviews
total_unpaid_1_star_reviews = unpaid_reviews_df.filter(col("star_rating") == 1).count()
print("Total number of unpaid 1 star reviews = %f" %total_unpaid_1_star_reviews)

unpaid_1_star_review_percentage = total_unpaid_1_star_reviews/total_unpaid_reviews
print("Percentage of unpaid 1 star reviews = %f" %unpaid_1_star_review_percentage)

Total number of unpaid 1 star reviews = 1532.000000
Percentage of unpaid 1 star reviews = 0.105823


In [39]:
paid_reviews_avg_star = ((total_paid_5_star_reviews*5)+(total_paid_4_star_reviews*4)+(total_paid_3_star_reviews*3)+(total_paid_2_star_reviews*2)+(total_paid_1_star_reviews*1))/total_paid_reviews
print("Average star rating for paid reviews: %f" %paid_reviews_avg_star)

Average star rating for paid reviews: 4.383333


In [42]:
unpaid_reviews_avg_star = ((total_unpaid_5_star_reviews*5)+(total_unpaid_4_star_reviews*4)+(total_unpaid_3_star_reviews*3)+(total_unpaid_2_star_reviews*2)+(total_unpaid_1_star_reviews*1))/total_unpaid_reviews
print("Average star rating for unpaid reviews: %f" %unpaid_reviews_avg_star)

Average star rating for unpaid reviews: 4.056365
